In [1]:
#!/usr/bin/env python
# coding: utf-8
#
# Author:   Kazuto Nakashima
# URL:      http://kazuto1011.github.io
# Created:  2017-05-26

from collections import OrderedDict

import cv2
from skimage import io
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim
import os
from skimage import io
import pandas as pd
import numpy as np
from skimage import img_as_float
from tqdm import tqdm

In [2]:
default_path = '/home/DL-based-Tumor-Classification/Datasets/trial_2nd_8843_true_feature'
final_conv = 'conv3'
num_epochs = 50
batch_size = 400
learning_rate = 0.0001

In [3]:
torch.cuda.empty_cache()
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
def preprocess(image_path):
    raw_image = cv2.imread(image_path,0)
#     raw_image = cv2.resize(raw_image, (96,) * 2)
    image = transforms.Compose(
        [
            transforms.ToTensor()        
        ]
    )(raw_image[..., ::-1].copy())
    image = image.type(torch.DoubleTensor)
    return image, raw_image

In [5]:
def get_device(cuda):
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device

In [3]:
class Net(nn.Module):

    def __init__(self, num_of_classes):
        super(Net, self).__init__()
        # input image channel, output channels, kernel size square convolution
        # kernel
        # input size = 102, output size = 100
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        # input size = 50, output size = 48
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        # input size = 24, output size = 24
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.vp = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(256*12*12, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, num_of_classes)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.bn1(self.vp(self.conv1(x))))
        x = F.relu(self.bn2(self.vp(self.conv2(x))))
        x = F.relu(self.bn3(self.vp(self.conv3(x))))
        x = self.drop2D(x)
        x = x.view(in_size, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x


net = Net(num_of_classes=2)
net = net.double()
# net = nn.DataParallel(net)
net.cuda()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2D): Dropout2d(p=0.25, inplace=False)
  (vp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=36864, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=2, bias=True)
)

In [4]:
def_path = default_path + '/img_fold8'
state_dict = torch.load(def_path + '/network_0505_th1.pth')
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
net.load_state_dict(new_state_dict)
net.eval()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2D): Dropout2d(p=0.25, inplace=False)
  (vp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=36864, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=2, bias=True)
)

In [5]:
train_csv_path = def_path + '/train/labels_train.csv'
train_root_path = def_path + '/train'
test_csv_path = def_path + '/test/labels_test.csv'
test_root_path = def_path + '/test'
test_label_path =  def_path + '/test/test_label_run_th1_test.csv'
predicted_label_path = def_path + '/test/predicted_label_run_th1_test.csv'
model_path = def_path + '/network_0505_th1.pth'


class TumorDatasetTrain(Dataset):

    def __init__(self, csv_file , root_dir, transform=None):
        self.labels_frame = np.array(pd.read_csv(csv_file, skiprows=1, sep=',', header=None)).astype(np.int)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = str(idx)+'.png'
        img_path = os.path.join(self.root_dir, img_name)
        """
            !!!Pay attention!!!
            The image size is set here
            """
        img = np.empty(shape=(1, 96, 96))
        img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
        label = np.array([self.labels_frame[idx,1]-1])
        train_sample = {'image': img, 'label': label}

        if self.transform:
            train_sample = self.transform(train_sample)
        return train_sample


class TumorDatasetTest(Dataset):

    def __init__(self, csv_file , root_dir, transform=None):
        self.labels_frame = np.array(pd.read_csv(csv_file, skiprows= 1, sep=',', header= None)).astype(np.int)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = str(idx) + '.png'
        img_path = os.path.join(self.root_dir, img_name)
        """
            !!!Pay attention!!!
            The image size is set here
            """
        img = np.empty(shape=(1, 96, 96))
        img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
        label = np.array([self.labels_frame[idx, 1]-1])
        test_sample = {'image': img, 'label': label}

        if self.transform:
            test_sample = self.transform(test_sample)
        return test_sample


class ToTensor(object):

    def __call__(self, sample):
        image, labels = sample['image'], sample['label']
        return {'image': torch.from_numpy(image), 'label': torch.LongTensor(labels)}


train_dataset = TumorDatasetTrain(csv_file=train_csv_path, root_dir=train_root_path,
                                  transform=transforms.Compose([ToTensor()]))
test_dataset = TumorDatasetTest(csv_file=test_csv_path, root_dir=test_root_path, transform=transforms.Compose([ToTensor()]))
dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)


In [6]:
class _PropagationBase(object):

    def __init__(self, model, cuda=True):
        self.model = model
        self.model.eval()
        if cuda:
            self.model.cuda()
        self.all_fmaps = OrderedDict()
        self.all_grads = OrderedDict()
        self._set_hook_func()
        self.image = None

    def _set_hook_func(self):
        raise NotImplementedError

    def _encode_one_hot(self, idx):
        one_hot = torch.DoubleTensor(1, self.preds.size()[-1]).zero_()
        one_hot[0][idx] = 1.0
        return one_hot.cuda()

#     def forward(self, image):
#         self.image = image
#         self.preds = self.model.forward(self.image)
#         self.probs = F.softmax(self.preds, dim=1)
#         return self.probs.sort(dim = 1, descending= True)

    def forward(self, image):
        self.image = image
        self.preds = self.model.forward(self.image)
        self.probs = F.softmax(self.preds, dim=1)[0]
        self.prob, self.idx = self.probs.data.sort(0, True)
        return self.prob, self.idx

    def backward(self, idx):
        self.model.zero_grad()
        one_hot = self._encode_one_hot(idx)
        self.preds.backward(gradient=one_hot, retain_graph=True)
        print("no error backward")

In [7]:
class GradCAM(_PropagationBase):

    def _set_hook_func(self):

        def func_f(module, input, output):
            self.all_fmaps[id(module)] = output.data.cpu()

        def func_b(module, grad_in, grad_out):
            self.all_grads[id(module)] = grad_out[0].cpu()

        for module in self.model.named_modules():
            module[1].register_forward_hook(func_f)
            module[1].register_backward_hook(func_b)

    def _find(self, outputs, target_layer):
        for key, value in outputs.items():
            for module in self.model.named_modules():
                if id(module[1]) == key:
                    if module[0] == target_layer:
                        return value
        raise ValueError('Invalid layer name: {}'.format(target_layer))

    def _normalize(self, grads):
        l2_norm = torch.sqrt(torch.mean(torch.pow(grads, 2))) + 1e-5
        # return grads / l2_norm.data[0]
        return grads / l2_norm.data[0]

    def _compute_grad_weights(self, grads):
        grads = self._normalize(grads)
        self.map_size = grads.size()[2:]
        return nn.AvgPool2d(self.map_size)(grads)

    def generate(self, target_layer):
        fmaps = self._find(self.all_fmaps, target_layer)
        grads = self._find(self.all_grads, target_layer)
        weights = self._compute_grad_weights(grads)
        gcam = torch.DoubleTensor(self.map_size).zero_()
        for fmap, weight in zip(fmaps[0], weights[0]):
            res = fmap * weight.data.expand_as(fmap)
            gcam += fmap * weight.data.expand_as(fmap)
        gcam = F.relu(Variable(gcam))

        gcam = gcam.data.cpu().numpy()
        gcam -= gcam.min()
        if gcam.max() != 0:
            gcam /= gcam.max()
        gcam = cv2.resize(gcam, (self.image.size(3), self.image.size(2)))

        return gcam

    def save(self, filename, gcam, raw_image):
        gcam = cv2.applyColorMap(np.uint8(gcam * 255.0), cv2.COLORMAP_JET)
        # gcam = gcam.astype(np.float) + raw_image.astype(np.float)
        if gcam.max() != 0:
            gcam = gcam / gcam.max() * 255.0
        cv2.imwrite(filename, np.uint8(gcam))

In [8]:
class BackPropagation(_PropagationBase):
    def _find(self, outputs, target_layer):
        for key, value in outputs.items():
            for module in self.model.named_modules():
                if id(module[1]) == key:
                    if module[0] == target_layer:
                        return value
        raise ValueError('Invalid layer name: {}'.format(target_layer))

    def generate(self):
        print('image_size:  ', self.image.size())
        print('image_grad:  ', self.image.grad)
        output = self.image.grad.data[0].cpu().numpy()[0]
        return output

    def save(self, filename, data):
        abs_max = np.maximum(-1 * data.min(), data.max())
        data = data / abs_max * 127.0 + 127.0
        cv2.imwrite(filename, np.uint8(data))

In [9]:
class GuidedBackPropagation(BackPropagation):

    def _set_hook_func(self):

        def func_b(module, grad_in, grad_out):
            print("mlaku kene func")
            self.all_grads[id(module)] = grad_in[0]

            # Cut off negative gradients
            if isinstance(module, nn.ReLU):
                print('mlaku po?')
                return torch.clamp(grad_in[0], min=0.0)

        for module in self.model.named_modules():
            module[1].register_backward_hook(func_b)

In [ ]:
# for ii, test_sample in enumerate(dataloader_test):
#         test_imgs = Variable(test_sample['image']).cuda()
#         print("SEE THIS::: " , test_imgs.shape)
#         test_label = Variable(test_sample['label']).cuda()
#         print(test_label.shape)
device = get_device(True)
image_paths = []
for filename in os.listdir(def_path + '/test'):
    directory = def_path + '/test/'
    if filename.endswith('.png'):
        image_paths.append(directory + filename)
images = []
raw_images = []
print("Images:")
for i, image_path in enumerate(image_paths):
    print("\t#{}: {}".format(i, image_path))
    image, raw_image = preprocess(image_path)
    images.append(image)
    raw_images.append(raw_image)
images = torch.stack(images).to(device)

In [14]:
gcam = GradCAM(model=net)
probs, ids = gcam.forward(images)

print(probs.shape , ids.shape)
#gbackprop
gbp = GuidedBackPropagation(model = net)
_ = gbp.forward(images)
print("ikisingmax ",torch.max(ids, 0))
print("ikisingmax ",torch.max(ids, 1))
for i in range(2):
    print("ikiids", ids[:,i])
    gbp.backward(idx=ids[:, [i]])
    gradients = gbp.generate()

    gcam.backward(idx=ids[:, [i]])
    regions = gcam.generate(target_layer = 'conv3')

    for i in range(len(images)):
#         features_blobs = []
#         print("now in loop "+str(i)+"!!!!!")
#         test_imgs_temp = test_imgs[i,:,:,:]
#         test_imgs_in = test_imgs_temp.unsqueeze(0)
#         test_label_temp = test_label[i,:]
#         test_label_in = test_label_temp.unsqueeze(0)
        gbp.save(filename = def_path + '/gradcamcoba/gbp/' + j +'_gbp.png', data = gradients[j])
        gcam.save(filename = def_path + '/gradcamcoba/gcam/' + j +'_gcam.png', gcam = regions[j,0], raw_image = raw_images[j])


torch.Size([65, 2]) torch.Size([65, 2])
ikisingmax  torch.return_types.max(
values=tensor([64, 64], device='cuda:0'),
indices=tensor([14, 50], device='cuda:0'))
ikisingmax  torch.return_types.max(
values=tensor([58, 40, 31, 55, 57, 43, 37, 22, 42, 12, 26, 28, 41, 35, 64, 16, 59, 50,
        33, 21, 61, 62, 54, 53, 32, 51, 48, 25, 60, 63, 30, 44, 56, 44, 30, 63,
        60, 25, 48, 51, 32, 53, 54, 62, 61, 21, 33, 50, 59, 16, 64, 35, 41, 28,
        26, 12, 42, 22, 37, 43, 57, 55, 31, 40, 58], device='cuda:0'),
indices=tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0], device='cuda:0'))
ikiids tensor([ 1, 40, 31, 39, 57, 43, 19, 22, 17,  8, 26, 28, 41, 14, 64,  9, 59, 50,
         2, 11, 18, 49, 54, 53,  4, 51, 10,  3, 52, 63, 30, 44, 56, 24, 29, 47,
        60, 25, 48, 45, 32, 38, 34, 62, 61, 21, 33, 36, 46, 16, 27, 35, 

IndexError: index 40 is out of bounds for dimension 0 with size 2

In [10]:
#gcam
for ii, test_sample in enumerate(dataloader_test):
        test_imgs = Variable(test_sample['image']).cuda()
        test_label = Variable(test_sample['label']).cuda()
#         label_coba = test_label.view(-1)
        
#         bp = BackPropagation(model=net)
#         probs, ids = bp.forwards(test_imgs)
        
       
#         print(probs)
#         print("ikisingmax ",torch.max(ids, 0))
#         print("ikisingmax ",torch.max(ids, 1))
        
        for j in range(0,test_imgs.size(0)):
            features_blobs = []
            print("now in loop "+str(j)+"!!!!!")
            test_imgs_temp = test_imgs[j,:,:,:]
            test_imgs_in = test_imgs_temp.unsqueeze(0)
            test_label_temp = test_label[j,:]
            test_label_in = test_label_temp.unsqueeze(0)
            
            gcam = GradCAM(model=net)
            probs, ids = gcam.forward(test_imgs_in)
            #gbackprop
            gbp = GuidedBackPropagation(model = net)
            probs, ids = gbp.forward(test_imgs_in)
            
            for i in range(0,2):
                print("ikiids", ids)
                gbp.backward(idx=ids[i])
                gradients = gbp.generate()

                gcam.backward(idx=ids[:, [i]])
                regions = gcam.generate(target_layer = 'conv3')
            
            gbp.save(filename = def_path + '/gradcamcoba/gbp/' + j +'_gbp.png', data = gradients[j])
            gcam.save(filename = def_path + '/gradcamcoba/gcam/' + j +'_gcam.png', gcam = regions[j,0], raw_image = test_imgs_in)

                

now in loop 0!!!!!
ikiids tensor([0, 1], device='cuda:0')
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
mlaku kene func
no error backward
image_size:   torch.Size([1, 1, 96, 96])
image_grad:   None


AttributeError: 'NoneType' object has no attribute 'data'